In [ ]:
import json
from os.path import join, dirname
from os import listdir
import re
import pandas as pd

from watson_developer_cloud import DocumentConversionV1
from watson_developer_cloud import AlchemyLanguageV1

In [ ]:
doc = "Dove.docx"
tmp = !pwd
dir = tmp[0] 

formats = ['doc','docx','pdf']

In [ ]:
credentials = {
    "url": "https://gateway.watsonplatform.net/document-conversion/api",
    "password": "tQDOCgBzcC0K",
    "username": "1a238d8d-f4d7-45eb-a5b5-2e1ff2ba8797",
}

api_key = "568d7754d14e108c87191b14bb23bec6623eee8a"

In [ ]:
document_conversion = DocumentConversionV1(
    username=credentials.get('username'),
    password=credentials.get('password'),
    version='2016-06-30')

In [ ]:
alchemy_language = AlchemyLanguageV1(api_key=api_key)
combined_operations = ['page-image', 'entity', 'keyword', 'title', 'author', 'taxonomy', 'concept', 'doc-emotion','quantity']

In [ ]:
def xConvert(doc):

    with open(join(dir, doc), 'rb') as document:
        config = {'conversion_target': DocumentConversionV1.ANSWER_UNITS}
        output = document_conversion.convert_document(document=document, config=config)
        #print(json.dumps(output, indent=2))
    
    return output

output = xConvert(doc)

In [ ]:
def xFilter(output):

    blocks = []
    for au in output.get('answer_units'):
        au_text = au.get('content')[0].get('text')
        if "viewab" in au_text.lower():
            #print(au_text)
            #print('\n')
            blocks.append(au_text)
    
    return blocks

blocks = xFilter(output)

In [ ]:
table = pd.DataFrame(columns=['document','valid','left context','quantity','right context','snippet'])
i = 0

for doc in listdir(dir):
    
    ext = doc.split(".")[-1]
    if ext not in formats:
        print("Unsupported format: %s" % (doc))
        continue
    
    output = xConvert(doc)
    blocks = xFilter(output)
    
    for block in blocks:
        output = alchemy_language.combined(text=block, extract=combined_operations)
        for entity in output.get('entities'):
            if entity.get('type') == 'Quantity':
                covered_text = entity.get('text')
                left_context = ''
                right_context = ''
                for m in re.finditer(covered_text, block):
                    start = m.start()
                    end = m.end()
                    if start >= 50:
                        left_context = block[start-50:start]
                    else:
                        left_context = block[:start]
                    end = start + len(covered_text)
                    if len(block) - end >= 50:
                        right_context = block[end:end+50]
                    else:
                        right_context = block[end:] 
                i = i + 1
                table.loc[i] = [doc,'',left_context, covered_text, right_context,block]
        
table.to_csv(join(dir, 'table.csv'))

In [ ]:
table.head(10)